In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = llm | parser


In [5]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {} # 특정 대화를 기록하기 위해 사용

def get_session_history(sessionId: str) -> BaseChatMessageHistory:
    if sessionId not in store: 
        store[sessionId] = InMemoryChatMessageHistory()
    return store[sessionId]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {
    "configurable": {
        "session_id": "chat"
    }
}


In [6]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage(content="사용자의 질문에 2문장 이내로 짧게 대답해"),
    HumanMessage(content="오늘은 피자를 먹어야지!"),
    AIMessage(content="정말 좋은 생각이야. 음료는 무엇으로 할 거야?"),
    HumanMessage(content="내일은 수영을 가야지!"),
    AIMessage(content="수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?"),
    HumanMessage(content="주말에는 영화를 보러 갈 거야!"),
    AIMessage(content="주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?"),
]

In [7]:
with_message_history.invoke(messages + 
  [HumanMessage(content="주말에 내가 뭐하러 간다고 했지?")],
  config=config
)


'주말에 영화를 보러 간다고 했어! 어떤 영화를 볼 예정이야?'

In [8]:
with_message_history.invoke(
    [HumanMessage(content="방금 내가 무슨 질문을 했지?")],
    config=config
)

'주말에 무슨 일 하러 간다고 했는지 물어봤어.'

In [9]:
with_message_history.invoke(
    [HumanMessage(content="수영은 언제가기로 했지?")],
    config=config
)

'수영은 내일 가기로 했어!'

In [10]:
import langchain
langchain.debug = False

In [11]:
config_01 = {
    "configurable": {
        "session_id": "chat_01"
    }
}

with_message_history.invoke(
    [HumanMessage(content="내일은 수영을가야지!")],
    config=config_01
)

'좋은 계획이네요! 수영은 체력에도 좋고 기분 전환에도 도움이 됩니다. 어떤 수영 연습을 할 계획이신가요?'

In [12]:
with_message_history.invoke(
    [HumanMessage(content="내일 뭐하러 간다고 했지?")],
    config=config_01
)

'내일 수영을 가기로 했다고 하셨죠! 수영 즐겁게 하세요!'

In [13]:
config_02= {
    "configurable": {
        "session_id": "chat_02"
    }
}

with_message_history.invoke(
    [HumanMessage(content="내일 뭐하러 간다고 했지?")],
    config=config_02
)

'죄송하지만, 저는 개인적인 일정을 기억하거나 추적할 수 없습니다. 내일 계획에 대해 말해주시면 도와드릴 수 있는 부분이 있을지도 모릅니다!'

In [14]:
with_message_history.invoke(
    [HumanMessage(content="언제간다고 했지?")],
    config=config_01
)

'내일 간다고 하셨습니다. 수영에 대한 구체적인 시간은 언급하지 않으셨는데, 혹시 시간도 정해두셨나요?'